<a href="https://colab.research.google.com/github/ssvadla/Legal_Text_Classification/blob/main/Co_training_3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive
import numpy as np
from sklearn.datasets import make_classification
import math
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.stem import WordNetLemmatizer
import nltk
import re
import string
import lightgbm as lgb
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
record_unlabel_list = []
record_f1_score = []
record_CR = []

In [ ]:
#Co-training Algorithm
def training(unlabel_1,train,i,j):
    train_size = len(train)
    record_unlabel_list.append(train_size - 4416)
    target_values = np.unique(train['Target'].values)

    data_list = []
    data_length_list = []
    for i in target_values:
      df_k = train[train['Target']==i]
      data_list.append(df_k)
      data_length_list.append(len(df_k))
   

    maximum_data = max(data_length_list)
    print("maximum_data",maximum_data)
    ratio = math.floor(( 4 / 6 )* maximum_data)
    print("ratio",ratio)
    loop_count = 0
    train_upsampled = []
    for i in data_length_list:
      #print(i)
      #print(len(data_list[loop_count]))
      if i < ratio:
        df_upsampled = resample(data_list[loop_count],replace=True,n_samples=ratio,random_state=123)
      else:
        df_upsampled = resample(data_list[loop_count],replace=True,n_samples=i,random_state=123)
      print("@",len(df_upsampled))
      train_upsampled.append(df_upsampled)
      loop_count = loop_count + 1

    df_res = pd.concat(train_upsampled)
    train = df_res.copy(deep = True)
    print("len of train $$$",len(train))

    i_list.append(i)
    j_list.append(j)
    print("length of the UL received",len(unlabel_1))
    
    #cleaning

    nltk.download('stopwords')
    nltk.download('wordnet')
    stopword=nltk.corpus.stopwords.words('english')

    wl= WordNetLemmatizer()

    def clean_text(text):
      text="".join([word.lower() for word in text if word not in string.punctuation])
      tokens = re.split('\W+',text)
      text = [wl.lemmatize(word) for word in tokens if word not in stopword]
      return text



    tfidf_vect = TfidfVectorizer(analyzer = clean_text)
    #X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
    X_tfidf = tfidf_vect.fit_transform(train['text'])
    print(X_tfidf.shape)

    #Read the test data
    test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')
    test['Target']=test['Target'].replace(['Others'],'Invalid')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
    test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
    from nltk.corpus import stopwords
    words = stopwords.words('english')
    test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
    t_p = tfidf_vect.transform(test['Sentence'])

    classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
    classifier_lgb = lgb.LGBMClassifier()
    classifier_svm = svm.LinearSVC(multi_class='ovr',class_weight='balanced')

    X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.20,random_state=42)
    X_train_whole = X_tfidf
    Y_train_whole = train['Target']

    classifier_rf.fit(X_train, Y_train)
    classifier_lgb.fit(X_train, Y_train)
    classifier_svm.fit(X_train, Y_train)

    val_pred_rf = classifier_rf.predict(x_val)
    val_pred_lgb = classifier_lgb.predict(x_val)
    val_pred_svm = classifier_svm.predict(x_val)

    Accuracy_score_rf = accuracy_score(y_val,val_pred_rf)
    print("Accuracy_score_rf ",Accuracy_score_rf)
    Accuracy_score_lgb = accuracy_score(y_val,val_pred_lgb)
    print("Accuracy_score_lgb ",Accuracy_score_lgb)
    Accuracy_score_svm = accuracy_score(y_val,val_pred_svm)
    print("Accuracy_score_svm ",Accuracy_score_svm)

    classification_report_test_rf = classification_report(y_val,val_pred_rf)
    print("classification report rf",classification_report_test_rf)
    classification_report_test_lgb = classification_report(y_val,val_pred_lgb)
    print("classification report lgb",classification_report_test_lgb)
    classification_report_test_svm = classification_report(y_val,val_pred_svm)
    print("classification report svm ",classification_report_test_svm)
    
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    cross_valid_rf = cross_val_score(classifier_rf,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    print("cross validation score RF",cross_valid_rf)
    cross_valid_lgb = cross_val_score(classifier_lgb,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    print("cross validation score LGB",cross_valid_lgb)
    cross_valid_svm = cross_val_score(classifier_svm,x_val ,y_val, scoring = 'f1_weighted', cv=cv, n_jobs=-1).mean()
    print("cross validation score SVM",cross_valid_svm)

    f1_score_rf = f1_score(y_val,val_pred_rf,average='weighted')
    print(" f1 score test rf ",f1_score_rf )
    f1_score_lgb = f1_score(y_val,val_pred_lgb, average='weighted')
    print(" f1 score test lgb ",f1_score_lgb )
    f1_score_svm = f1_score(y_val,val_pred_svm, average='weighted')
    print(" f1 score test svm ",f1_score_svm )

    if (Accuracy_score_rf > Accuracy_score_lgb) and (Accuracy_score_rf > Accuracy_score_svm):
      B = classifier_rf
    elif (Accuracy_score_svm > Accuracy_score_lgb) and (Accuracy_score_svm > Accuracy_score_rf):
      B = classifier_svm
    else:
      B = classifier_lgb

    B.fit(X_train_whole,Y_train_whole)
    print("len of train tfid #####",len(Y_train_whole))
    test_pred = B.predict(t_p)
    accuracy_test = accuracy_score(test['Target'],test_pred)
    print('Accuracy test data', accuracy_test)
    f1_score_test = f1_score(test['Target'],test_pred, average='weighted')
    print(" f1 score test ",f1_score_test )
    
    f1_score_test_list.append(f1_score_test)
    record_f1_score.append(f1_score_test)
    classification_report_test_data = classification_report(test['Target'],test_pred)
    record_CR.append(classification_report_test_data)
    print("classification report ",classification_report_test_data)
    

    class_x_un1 = tfidf_vect.transform(unlabel_1['text'])
    class_pred_unlabel_1 = B.predict(class_x_un1)
    unlabel_1['Target']=class_pred_unlabel_1
    frame_1 = [train,unlabel_1]
    train_1 = pd.concat(frame_1)
       
    with open(r'/content/drive/My Drive/Results/Co_Training_3C/Result01.txt', 'a') as writefile:
        writefile.write("\n")
        writefile.write(" Co-training results ")
        writefile.write("\n")
        writefile.write(" train_size ")
        writefile.write("\n")
        writefile.write(str(train_size))
        writefile.write(" Accuracy_score_rf = ")
        writefile.write(str(Accuracy_score_rf))
        writefile.write(" Accuracy_score_lgb = ")
        writefile.write(str(Accuracy_score_lgb))
        writefile.write("\n")
        writefile.write(" Accuracy_score_svm = ")
        writefile.write(str(Accuracy_score_svm))
        writefile.write("\n")
        writefile.write(" classification_report_test_rf = ")
        writefile.write("\n")
        writefile.write(str(classification_report_test_rf))
        writefile.write("\n")
        writefile.write(" classification_report_test_lgb = ")
        writefile.write("\n")
        writefile.write(str(classification_report_test_lgb))
        writefile.write("\n")
        writefile.write(" classification_report_test_svm = ")
        writefile.write("\n")
        writefile.write(str(classification_report_test_svm))
        writefile.write("\n")
      
        writefile.write(" cross_valid_rf = ")
        writefile.write(str(cross_valid_rf))
        writefile.write("\n")
        writefile.write(" cross_valid_lgb = ")
        writefile.write(str(cross_valid_lgb))
        writefile.write("\n")
        writefile.write(" cross_valid_svm = ")
        writefile.write(str(cross_valid_svm))
        writefile.write("\n")
        
        writefile.write(" f1_score_rf = ")
        writefile.write(str(f1_score_rf))
        writefile.write("\n")
        writefile.write(" f1_score_lgb = ")
        writefile.write(str(f1_score_lgb))
        writefile.write("\n")
        writefile.write(" f1_score_svm = ")
        writefile.write(str(f1_score_svm))
        writefile.write("\n")
        writefile.write(" Results on test data ")
        writefile.write("\n")
        writefile.write(" f1_score_test = ")
        writefile.write(str(f1_score_test))
        writefile.write("\n")
        writefile.write(" classifier selected = ")
        writefile.write(str(B))
        writefile.write("\n")
        writefile.write(" classification_report_test_data = ")
        writefile.write(str(classification_report_test_data))
        writefile.write("\n")
    
    return train_1,train_size_list,f1_score_test_list


In [ ]:
import pandas as pd

In [ ]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Read the unlabeled data
unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


537703

In [ ]:

#Read the labeled data
train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
    #print(i)
    train = train.append(i)

train.sort_values("Sentence", inplace = True)
train = train.drop_duplicates(subset ="Sentence")
train['Target'].unique()
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()
print(len(train))
train = train.rename(columns={'Sentence':'text'})
all_Data_train_size = len(train)

37711
4416


In [ ]:

length_list = [500]

In [ ]:
j=0
iteration = 0
i_list = []
j_list = []
f1_score_test_list = []
train_size_list =[]

for i in length_list:
  print("iteration ",iteration)
  print(i)
  print(j)
  unlabel_chunk = unlabel[j:i]
  train,op_train_size_list,op_f1_score_test_list = training(unlabel_chunk,train,i,j)
  j=i
  iteration = iteration + 1
  print("length",len(unlabel_chunk))

#Write into text file:    
with open(r'/content/drive/My Drive/Results/Co_Training_3C/Result01.txt', 'a') as writefile:
    writefile.write("\n")
    writefile.write(" Co-training results ")
    writefile.write("\n")
    writefile.write("\n")
    writefile.write(" train size ")
    writefile.write(str(all_Data_train_size))
    writefile.write("\n")
    writefile.write("\n")
    writefile.write("op_train_size_list")
    writefile.write("\n")
    writefile.write(str(op_train_size_list))
    writefile.write("\n")
    writefile.write("op_f1_score_test_list")
    writefile.write("\n")
    writefile.write(str(op_f1_score_test_list))
    


iteration  0
500
0
maximum_data 2253
ratio 1502
@ 1502
@ 1502
@ 2253
@ 1502
@ 1502
@ 1502
len of train $$$ 9763
length of the UL received 500
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
(9763, 6515)
Accuracy_score_rf  0.9482846902201741
Accuracy_score_lgb  0.8965693804403482
Accuracy_score_svm  0.9339477726574501
classification report rf                   precision    recall  f1-score   support

        Analysis       0.94      0.90      0.92       321
      Conclusion       0.97      0.98      0.98       299
           Facts       0.94      0.90      0.92       435
         Invalid       0.89      0.96      0.92       294
           Issue       0.98      1.00      0.99       295
Rule/Law/Holding       0.98      0.97      0.98       309

        accuracy                           0.95      1953
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
with open(r'/content/drive/My Drive/Results/Co_Training_3C/Result01.txt', 'a') as writefile:
    writefile.write("\n")
    writefile.write(" Co-training results ")
    writefile.write("\n")
    writefile.write("\n")
    writefile.write(" train size ")
    writefile.write(str(all_Data_train_size))
    writefile.write("\n")
    writefile.write("\n")
    writefile.write("op_train_size_list")
    writefile.write("\n")
    writefile.write(str(op_train_size_list))
    writefile.write("\n")
    writefile.write("op_f1_score_test_list")
    writefile.write("\n")
    writefile.write(str(op_f1_score_test_list))
    

In [ ]:
with open(r'/content/drive/My Drive/Results/Co_Training_3C/Result02.txt', 'a') as writefile:

  writefile.write("\n")
  writefile.write("record_unlabel_list")
  writefile.write("\n")
  writefile.write(str(record_unlabel_list))
  writefile.write("\n")
  writefile.write("record_f1_score")
  writefile.write("\n")
  writefile.write(str(record_f1_score))
  writefile.write("\n")
  writefile.write("record_CR")
  writefile.write("\n")
  writefile.write(str(record_CR))
